In [1]:
library(broom)
library(scales)
library(stringr)
library(arrow)
library(survival)
library(tidyr)
library(dplyr)
library(haven)
library(reshape2)
library(tidyverse)
library(data.table)
library(lubridate)
library(ggplot2)

Warning message:
“package ‘arrow’ was built under R version 4.4.1”

Attaching package: ‘arrow’


The following object is masked from ‘package:utils’:

    timestamp



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ forcats   1.0.0     ✔ purrr     1.0.2
✔ ggplot2   3.5.1     ✔ readr     2.1.5
✔ lubridate 1.9.3     ✔ tibble    3.2.1
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ readr::col_factor()   masks scales::col_factor()
✖ purrr::discard()      masks scales::discard()
✖ lubridate::duration() masks arrow::duration()
✖ dplyr::filter()       masks stats::filter()
✖ dplyr::lag()          masks stats::lag()
ℹ Use the conf

In [2]:
x<- load("data_all_pheno_cox3_UKB.Rdata")
x
target_mca_types <- colnames(data_all_pheno_cox3)[4842:4859]
target_mca_types

[1] "data_all_pheno_cox3"

[1] "chr11p.CN.LOH" "chr11q.CN.LOH" "chr12.Gain"    "chr13q.CN.LOH"
 [5] "chr13q.Loss"   "chr14q.CN.LOH" "chr15.CN.LOH"  "chr15.Gain"   
 [9] "chr17q.CN.LOH" "chr1p.CN.LOH"  "chr1q.CN.LOH"  "chr20q.Loss"  
[13] "chr22q.CN.LOH" "chr3q.CN.LOH"  "chr4q.CN.LOH"  "chr6p.CN.LOH" 
[17] "chr9p.CN.LOH"  "chr9q.CN.LOH"

In [3]:
disease_cols <- colnames(data_all_pheno_cox3)[50:2444]
disease_cols

[1] "(Idiopathic) normal pressure hydrocephalus"                                                    
   [2] "46, XX/46, XY*"                                                                                
   [3] "Abdominal aortic aneurysm"                                                                     
   [4] "Abdominal pain"                                                                                
   [5] "Abnormal digit morphology"                                                                     
   [6] "Abnormal findings on antenatal screening of mother"                                            
   [7] "Abnormal forces of labor"                                                                      
   [8] "Abnormal glucose"                                                                              
   [9] "Abnormal intraocular pressure"                                                                 
  [10] "Abnormal involuntary movements"                                                                
  [11] "Abnormal products of conception"                                                               
  [12] "Abnormalities of breathing"                                                                    
  [13] "Abnormalities of ear ossicles"                                                                 
  [14] "Abnormalities of gravid uterus"                                                                
  [15] "Abnormalities of size and form of teeth"                                                       
  [16] "Abnormalities of the hair"                                                                     
  [17] "Abnormality of gait and mobility"                                                              
  [18] "Abnormality of the lymph nodes"                                                                
  [19] "Abnormality of the peritoneum"                                                                 
  [20] "Abnormality of the urachus"                                                                    
  [21] "Abnormality of urination"                                                                      
  [22] "ABO isoimmunization of newborn"                                                                
  [23] "Abscess of liver"                                                                              
  [24] "Abscess of lung and mediastinum"                                                               
  [25] "Abscess of spleen*"                                                                            
  [26] "Abscess or ulceration of vulva"                                                                
  [27] "Absence of iris [Aniridia]"                                                                    
  [28] "Abuse of non-psychoactive substances*"                                                         
  [29] "Acantholytic Disorders*"                                                                       
  [30] "Acanthosis nigricans"                                                                          
  [31] "Accessory auricle"                                                                             
  [32] "Accessory kidney*"                                                                             
  [33] "Accessory lobe of lung*"                                                                       
  [34] "Achalasia of cardia"                                                                           
  [35] "Achilles tendinitis"                                                                           
  [36] "Achondroplasia*"                                                                               
  [37] "Acidosis"                                                                                      
  [38] "Acne"                                                                                          
  [39] "Acquired atrophy of ovary and fallopian tub

In [4]:
target_columns <- data_all_pheno_cox3[, 50:2444]
freq_counts <- colSums(target_columns != 0, na.rm = TRUE)
low_frequency_columns <- freq_counts[freq_counts <= 10]
cat("Number of columns with frequency < 50:", length(low_frequency_columns), "\n")
low_frequency_columns <- names(low_frequency_columns)
length(low_frequency_columns)
disease_cols <- setdiff(disease_cols, low_frequency_columns)
length(disease_cols)

Number of columns with frequency < 50: 522 


[1] 522

[1] 1873

In [5]:
cox_results <- list()

In [5]:
for (i in seq_along(disease_cols)) {
  disease_col <- disease_cols[i]
  time_col <- paste0("time_", disease_col)

  cat("Processing:", time_col, "（Loop", i, "of", length(disease_cols), "）\n")
    
  data_surv <- data_all_pheno_cox3 %>%
    mutate(
      surv_time = case_when(
        !!sym(disease_col) == 1 ~ as.numeric(difftime(!!sym(time_col), min_date, units = "days")) / 365.25,
        !!sym(disease_col) == 0 ~ as.numeric(difftime(max_date, min_date, units = "days")) / 365.25
      )
    ) %>%
    filter(surv_time > 0)
  
  total_samples <- sum(data_surv[[disease_col]] == 1)

  surv_obj <- Surv(time = data_surv$surv_time, event = data_surv[[disease_col]])
  
  cox_model <- coxph(surv_obj ~ chr11q.CN.LOH + baseline_age + age2 + smoking_0 + genetic_sex + PC1 + PC2 + PC3 + PCD4 + PC5, data = data_surv)
  
  cox_result <- tidy(cox_model) %>%
    filter(term == "chr11q.CN.LOH") %>%
    mutate(
      total_samples = total_samples,
      log_p_value = -log10(p.value)
    )
  
  cox_results[[disease_col]] <- cox_result
}


results_df <- bind_rows(cox_results, .id = "disease")
results_df <- results_df %>% arrange(desc(log_p_value))
write.csv(results_df, "chr11qCN.LOH_UKB_results.csv", row.names = FALSE)

Processing: time_Abdominal aortic aneurysm （Loop 1 of 1469 ）
Processing: time_Abdominal pain （Loop 2 of 1469 ）
Processing: time_Abnormal glucose （Loop 3 of 1469 ）


Warning message in coxph.fit(X, Y, istrat, offset, init, control, weights = weights, :
“Loglik converged before variable  1 ; coefficient may be infinite. ”


Processing: time_Abnormal intraocular pressure （Loop 4 of 1469 ）
Processing: time_Abnormal involuntary movements （Loop 5 of 1469 ）
Processing: time_Abnormal products of conception （Loop 6 of 1469 ）


Warning message in coxph.fit(X, Y, istrat, offset, init, control, weights = weights, :
“Loglik converged before variable  1 ; coefficient may be infinite. ”


Processing: time_Abnormality of gait and mobility （Loop 7 of 1469 ）


...

Processing: time_Wegener's granulomatosis （Loop 1467 of 1469 ）


Warning message in coxph.fit(X, Y, istrat, offset, init, control, weights = weights, :
“Loglik converged before variable  1 ; coefficient may be infinite. ”


Processing: time_Xanthelasma of eyelid （Loop 1468 of 1469 ）


Warning message in coxph.fit(X, Y, istrat, offset, init, control, weights = weights, :
“Loglik converged before variable  1 ; coefficient may be infinite. ”


Processing: time_Xerosis cutis* （Loop 1469 of 1469 ）


In [6]:
for (i in seq_along(disease_cols)) {
  disease_col <- disease_cols[i]
  time_col <- paste0("time_", disease_col)

  cat("Processing:", time_col, "（Loop", i, "of", length(disease_cols), "）\n")
    
  data_surv <- data_all_pheno_cox3 %>%
    mutate(
      surv_time = case_when(
        !!sym(disease_col) == 1 ~ as.numeric(difftime(!!sym(time_col), min_date, units = "days")) / 365.25,
        !!sym(disease_col) == 0 ~ as.numeric(difftime(max_date, min_date, units = "days")) / 365.25
      )
    ) %>%
    filter(surv_time > 0)
  
  total_samples <- sum(data_surv[[disease_col]] == 1)

  surv_obj <- Surv(time = data_surv$surv_time, event = data_surv[[disease_col]])
  
  cox_model <- coxph(surv_obj ~ chr13q.Loss + baseline_age + age2 + smoking_0 + genetic_sex + PC1 + PC2 + PC3 + PCD4 + PC5, data = data_surv)
  
  cox_result <- tidy(cox_model) %>%
    filter(term == "chr13q.Loss") %>%
    mutate(
      total_samples = total_samples,
      log_p_value = -log10(p.value)
    )
  
  cox_results[[disease_col]] <- cox_result
}


results_df <- bind_rows(cox_results, .id = "disease")
results_df <- results_df %>% arrange(desc(log_p_value))
write.csv(results_df, "chr13qLoss_UKB_results.csv", row.names = FALSE)

Processing: time_Abdominal aortic aneurysm （Loop 1 of 1469 ）
Processing: time_Abdominal pain （Loop 2 of 1469 ）
Processing: time_Abnormal glucose （Loop 3 of 1469 ）


Warning message in coxph.fit(X, Y, istrat, offset, init, control, weights = weights, :
“Loglik converged before variable  1 ; coefficient may be infinite. ”


Processing: time_Abnormal intraocular pressure （Loop 4 of 1469 ）
Processing: time_Abnormal involuntary movements （Loop 5 of 1469 ）
Processing: time_Abnormal products of conception （Loop 6 of 1469 ）


Warning message in coxph.fit(X, Y, istrat, offset, init, control, weights = weights, :
“Loglik converged before variable  1 ; coefficient may be infinite. ”


Processing: time_Abnormality of gait and mobility （Loop 7 of 1469 ）


...

Processing: time_Wegener's granulomatosis （Loop 1467 of 1469 ）


Warning message in coxph.fit(X, Y, istrat, offset, init, control, weights = weights, :
“Loglik converged before variable  1 ; coefficient may be infinite. ”


Processing: time_Xanthelasma of eyelid （Loop 1468 of 1469 ）


Warning message in coxph.fit(X, Y, istrat, offset, init, control, weights = weights, :
“Loglik converged before variable  1 ; coefficient may be infinite. ”


Processing: time_Xerosis cutis* （Loop 1469 of 1469 ）


In [7]:
for (i in seq_along(disease_cols)) {
  disease_col <- disease_cols[i]
  time_col <- paste0("time_", disease_col)

  cat("Processing:", time_col, "（Loop", i, "of", length(disease_cols), "）\n")
    
  data_surv <- data_all_pheno_cox3 %>%
    mutate(
      surv_time = case_when(
        !!sym(disease_col) == 1 ~ as.numeric(difftime(!!sym(time_col), min_date, units = "days")) / 365.25,
        !!sym(disease_col) == 0 ~ as.numeric(difftime(max_date, min_date, units = "days")) / 365.25
      )
    ) %>%
    filter(surv_time > 0)
  
  total_samples <- sum(data_surv[[disease_col]] == 1)

  surv_obj <- Surv(time = data_surv$surv_time, event = data_surv[[disease_col]])
  
  cox_model <- coxph(surv_obj ~ chr17q.CN.LOH + baseline_age + age2 + smoking_0 + genetic_sex + PC1 + PC2 + PC3 + PCD4 + PC5, data = data_surv)
  
  cox_result <- tidy(cox_model) %>%
    filter(term == "chr17q.CN.LOH") %>%
    mutate(
      total_samples = total_samples,
      log_p_value = -log10(p.value)
    )
  
  cox_results[[disease_col]] <- cox_result
}


results_df <- bind_rows(cox_results, .id = "disease")
results_df <- results_df %>% arrange(desc(log_p_value))
write.csv(results_df, "chr17qCNLOH_UKB_results.csv", row.names = FALSE)

Processing: time_Abdominal aortic aneurysm （Loop 1 of 1469 ）
Processing: time_Abdominal pain （Loop 2 of 1469 ）
Processing: time_Abnormal glucose （Loop 3 of 1469 ）


Warning message in coxph.fit(X, Y, istrat, offset, init, control, weights = weights, :
“Loglik converged before variable  1 ; coefficient may be infinite. ”


Processing: time_Abnormal intraocular pressure （Loop 4 of 1469 ）
Processing: time_Abnormal involuntary movements （Loop 5 of 1469 ）
Processing: time_Abnormal products of conception （Loop 6 of 1469 ）


Warning message in coxph.fit(X, Y, istrat, offset, init, control, weights = weights, :
“Loglik converged before variable  1 ; coefficient may be infinite. ”


Processing: time_Abnormality of gait and mobility （Loop 7 of 1469 ）


...

Warning message in coxph.fit(X, Y, istrat, offset, init, control, weights = weights, :
“Loglik converged before variable  1 ; coefficient may be infinite. ”


Processing: time_Xanthelasma of eyelid （Loop 1468 of 1469 ）


Warning message in coxph.fit(X, Y, istrat, offset, init, control, weights = weights, :
“Loglik converged before variable  1 ; coefficient may be infinite. ”


Processing: time_Xerosis cutis* （Loop 1469 of 1469 ）


Warning message in coxph.fit(X, Y, istrat, offset, init, control, weights = weights, :
“Loglik converged before variable  1 ; coefficient may be infinite. ”


In [6]:
for (i in seq_along(disease_cols)) {
  disease_col <- disease_cols[i]
  time_col <- paste0("time_", disease_col)

  cat("Processing:", time_col, "（Loop", i, "of", length(disease_cols), "）\n")
    
  data_surv <- data_all_pheno_cox3 %>%
    mutate(
      surv_time = case_when(
        !!sym(disease_col) == 1 ~ as.numeric(difftime(!!sym(time_col), min_date, units = "days")) / 365.25,
        !!sym(disease_col) == 0 ~ as.numeric(difftime(max_date, min_date, units = "days")) / 365.25
      )
    ) %>%
    filter(surv_time > 0)
  
  total_samples <- sum(data_surv[[disease_col]] == 1)

  surv_obj <- Surv(time = data_surv$surv_time, event = data_surv[[disease_col]])
  
  cox_model <- coxph(surv_obj ~ chr1q.CN.LOH + baseline_age + age2 + smoking_0 + genetic_sex + PC1 + PC2 + PC3 + PCD4 + PC5, data = data_surv)
  
  cox_result <- tidy(cox_model) %>%
    filter(term == "chr1q.CN.LOH") %>%
    mutate(
      total_samples = total_samples,
      log_p_value = -log10(p.value)
    )
  
  cox_results[[disease_col]] <- cox_result
}


results_df <- bind_rows(cox_results, .id = "disease")
results_df <- results_df %>% arrange(desc(log_p_value))
write.csv(results_df, "chr1qCNLOH_UKB_results.csv", row.names = FALSE)

Processing: time_(Idiopathic) normal pressure hydrocephalus （Loop 1 of 1873 ）


Warning message in coxph.fit(X, Y, istrat, offset, init, control, weights = weights, :
“Loglik converged before variable  1 ; coefficient may be infinite. ”


Processing: time_Abdominal aortic aneurysm （Loop 2 of 1873 ）
Processing: time_Abdominal pain （Loop 3 of 1873 ）
Processing: time_Abnormal digit morphology （Loop 4 of 1873 ）


Warning message in coxph.fit(X, Y, istrat, offset, init, control, weights = weights, :
“Loglik converged before variable  1 ; coefficient may be infinite. ”


Processing: time_Abnormal findings on antenatal screening of mother （Loop 5 of 1873 ）
Processing: time_Abnormal forces of labor （Loop 6 of 1873 ）


...

Warning message in coxph.fit(X, Y, istrat, offset, init, control, weights = weights, :
“Loglik converged before variable  1 ; coefficient may be infinite. ”


Processing: time_Xanthelasma of eyelid （Loop 1872 of 1873 ）


Warning message in coxph.fit(X, Y, istrat, offset, init, control, weights = weights, :
“Loglik converged before variable  1 ; coefficient may be infinite. ”


Processing: time_Xerosis cutis* （Loop 1873 of 1873 ）


Warning message in coxph.fit(X, Y, istrat, offset, init, control, weights = weights, :
“Loglik converged before variable  1 ; coefficient may be infinite. ”


In [ ]:
for (i in seq_along(disease_cols)) {
  disease_col <- disease_cols[i]
  time_col <- paste0("time_", disease_col)

  cat("Processing:", time_col, "（Loop", i, "of", length(disease_cols), "）\n")
    
  data_surv <- data_all_pheno_cox3 %>%
    mutate(
      surv_time = case_when(
        !!sym(disease_col) == 1 ~ as.numeric(difftime(!!sym(time_col), min_date, units = "days")) / 365.25,
        !!sym(disease_col) == 0 ~ as.numeric(difftime(max_date, min_date, units = "days")) / 365.25
      )
    ) %>%
    filter(surv_time > 0)
  
  total_samples <- sum(data_surv[[disease_col]] == 1)

  surv_obj <- Surv(time = data_surv$surv_time, event = data_surv[[disease_col]])
  
  cox_model <- coxph(surv_obj ~ chr6p.CN.LOH + baseline_age + age2 + smoking_0 + genetic_sex + PC1 + PC2 + PC3 + PCD4 + PC5, data = data_surv)
  
  cox_result <- tidy(cox_model) %>%
    filter(term == "chr6p.CN.LOH") %>%
    mutate(
      total_samples = total_samples,
      log_p_value = -log10(p.value)
    )
  
  cox_results[[disease_col]] <- cox_result
}


results_df <- bind_rows(cox_results, .id = "disease")
results_df <- results_df %>% arrange(desc(log_p_value))
write.csv(results_df, "chr6pCNLOH_UKB_results.csv", row.names = FALSE)

Processing: time_(Idiopathic) normal pressure hydrocephalus （Loop 1 of 1873 ）


Warning message in coxph.fit(X, Y, istrat, offset, init, control, weights = weights, :
“Loglik converged before variable  1 ; coefficient may be infinite. ”


Processing: time_Abdominal aortic aneurysm （Loop 2 of 1873 ）


Warning message in coxph.fit(X, Y, istrat, offset, init, control, weights = weights, :
“Loglik converged before variable  1 ; coefficient may be infinite. ”


Processing: time_Abdominal pain （Loop 3 of 1873 ）
Processing: time_Abnormal digit morphology （Loop 4 of 1873 ）


...

Warning message in coxph.fit(X, Y, istrat, offset, init, control, weights = weights, :
“Loglik converged before variable  1 ; coefficient may be infinite. ”


Processing: time_Disease of anus and rectum （Loop 570 of 1873 ）


Warning message in coxph.fit(X, Y, istrat, offset, init, control, weights = weights, :
“Loglik converged before variable  1 ; coefficient may be infinite. ”


Processing: time_Disease of capillaries （Loop 571 of 1873 ）


Warning message in coxph.fit(X, Y, istrat, offset, init, control, weights = weights, :
“Loglik converged before variable  1 ; coefficient may be infinite. ”
